In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [2]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [4]:
label_encoder = LabelEncoder()
data['Gender'] = label_encoder.fit_transform(data['Gender'])

In [9]:
one_hot_encoder =OneHotEncoder()
geography_encoded = one_hot_encoder.fit_transform(data[['Geography']]).toarray()
geography_encoded_df = pd.DataFrame(geography_encoded, columns=one_hot_encoder.get_feature_names_out())
geography_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [10]:
data = pd.concat([data.drop(['Geography'], axis=1), geography_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [11]:
# Splitting the dataset into features and target variable
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

# Splitting into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
scaler_reg = StandardScaler()
X_train = scaler_reg.fit_transform(X_train)
X_test = scaler_reg.transform(X_test)

# Save the scaler for future use
with open('scaler_reg.pkl', 'wb') as f:
    pickle.dump(scaler_reg, f)

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

c:\Users\FPK1COB\Documents\Learning\ML-assignments\ML-Assignments\venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [15]:
X_train.shape

(8000, 12)

12 -> 64 -> 32 -> 1

In [26]:
model = Sequential(
    [
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1)  # Output layer for regression
    ]
)

model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

model.summary()

c:\Users\FPK1COB\Documents\Learning\ML-assignments\ML-Assignments\venv\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [27]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [28]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [29]:
history = model.fit(
    X_train, y_train, 
    epochs=100,
    validation_data=(X_test, y_test),
    validation_split=0.2, 
    callbacks=[early_stopping_callback]
)

Epoch 1/100


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 100381.4375 - mae: 100381.4375 - val_loss: 98535.5156 - val_mae: 98535.5156
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 99717.4922 - mae: 99717.4922 - val_loss: 97193.4297 - val_mae: 97193.4297
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 97389.6797 - mae: 97389.6797 - val_loss: 93767.8750 - val_mae: 93767.8750
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 92811.7891 - mae: 92811.7891 - val_loss: 88026.4141 - val_mae: 88026.4141
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 86061.5156 - mae: 86061.5156 - val_loss: 80428.0078 - val_mae: 80428.0078
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 77791.7344 - mae: 77791.7344 - val_loss: 72189.2031 - val_mae: 72189.2031
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 69349.2109 - mae: 69349.2109 - val_loss: 64546.7266 - val_mae: 64546.7266
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 6200

Model with batch normalization

In [35]:
model_bnorm = Sequential(
    [
        Dense(64, input_shape=(X_train.shape[1],)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        Dense(32, activation='relu'),
        Dense(1)  # Output layer for regression
    ]
)

model_bnorm.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

c:\Users\FPK1COB\Documents\Learning\ML-assignments\ML-Assignments\venv\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [36]:
history = model_bnorm.fit(
    X_train, y_train, 
    epochs=100,
    validation_data=(X_test, y_test),
    validation_split=0.2, 
    callbacks=[early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 100367.7109 - mae: 100367.7109 - val_loss: 98514.7422 - val_mae: 98514.7422
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 99703.9453 - mae: 99703.9453 - val_loss: 97160.8438 - val_mae: 97160.8438
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 97545.8203 - mae: 97545.8203 - val_loss: 93892.1016 - val_mae: 93892.1016
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 93370.7109 - mae: 93370.7109 - val_loss: 88376.2812 - val_mae: 88376.2812
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 87150.5078 - mae: 87150.5078 - val_loss: 81175.2266 - val_mae: 81175.2266
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 79395.4062 - mae: 79395.4062 - val_loss: 73015.6719 - val_mae: 73015.6719
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 70863.3125 - mae: 70863.3125 - val_loss: 64860.0000 - val_mae: 64860.0000
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 

Batch Norm + dropout

In [37]:
model_bnorm_drp = Sequential(
    [
        Dense(64, input_shape=(X_train.shape[1],)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1)  # Output layer for regression
    ]
)

model_bnorm_drp.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

c:\Users\FPK1COB\Documents\Learning\ML-assignments\ML-Assignments\venv\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [38]:
history = model_bnorm_drp.fit(
    X_train, y_train, 
    epochs=100,
    validation_data=(X_test, y_test),
    validation_split=0.2, 
    callbacks=[early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 100391.1406 - mae: 100391.1406 - val_loss: 98585.0469 - val_mae: 98585.0469
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 99930.5156 - mae: 99930.5156 - val_loss: 97629.8203 - val_mae: 97629.8203
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 98369.7656 - mae: 98369.7656 - val_loss: 95250.2344 - val_mae: 95250.2344
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 95248.7969 - mae: 95248.7969 - val_loss: 91108.7812 - val_mae: 91108.7812
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 90471.3047 - mae: 90471.3047 - val_loss: 85464.4531 - val_mae: 85464.4531
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 84256.3828 - mae: 84256.3828 - val_loss: 78721.5156 - val_mae: 78721.5156
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 76864.5938 - mae: 76864.5938 - val_loss: 71062.1953 - val_mae: 71062.1953
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 